In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

sys.path.append("..")
from otc.metrics.metrics import effective_spread

import wandb

from tqdm.auto import tqdm

from scipy.stats import wilcoxon
import warnings


/pfs/data5/home/kit/stud/uloak/thesis/thesis/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# set here globally
exchange = "ise"
models = ["gbm", "classical"] # ["gbm", "classical"]
subset = "test" # "all"
strategy = "supervised"  # "transfer"


In [3]:
key = f"{exchange}_{strategy}_{subset}"
dataset = f"fbv/thesis/{exchange}_{strategy}_raw:latest"

os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

run = wandb.init(project="thesis", entity="fbv")

# load unscaled data
artifact = run.use_artifact(dataset)  # type: ignore
data_dir = artifact.download()

# load results
result_dirs = []
for model in models:
    results = f"fbv/thesis/{exchange}_{model}_{strategy}_{subset}:latest"
    artifact = run.use_artifact(results)  # type: ignore
    result_dir = artifact.download()
    result_dirs.append(result_dir)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karelze (fbv). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact ise_supervised_raw:latest, 2589.45MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact ise_gbm_supervised_test:latest, 129.80MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact ise_classical_supervised_test:latest, 278.79MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0


In [4]:
# p. 35-38
columns = [
    "buy_sell",
    "EXPIRATION",
    "QUOTE_DATETIME",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "ask_ex",
    "bid_ex",
    "myn",
    "OPTION_TYPE",
    "issue_type",
]


if subset == "all":
    train = pd.read_parquet(
        Path(data_dir, "train_set"), engine="fastparquet", columns=columns
    )
    val = pd.read_parquet(
        Path(data_dir, "val_set"), engine="fastparquet", columns=columns
    )
    test = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=columns
    )
    eval_data = pd.concat([train, val, test])
    del train, val, test

elif subset == "test":
    eval_data = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=columns
    )


results = []
for i, model in tqdm(enumerate(models)):
    result = pd.read_parquet(Path(result_dirs[i], "results"), engine="fastparquet")
    result.columns = pd.MultiIndex.from_product([[model], result.columns])
    results.append(result)

results_data = pd.concat(results, axis=1, names=models)

assert len(eval_data) == len(results_data)

X_print = eval_data

del results


2it [00:02,  1.27s/it]


In [5]:
# FIXME: select a subset of results for testing.
results_data = results_data[
    [
        ("gbm", "gbm(classical)"),
        ("gbm",'gbm(classical-retraining)'),
        ("gbm",'gbm(classical-size-retraining)'), 
        ("gbm", "gbm(classical-size)"),
        ("gbm", "gbm(ml)"),
        ("gbm",'gbm(ml-retraining)'),
        ("gbm",'gbm(semi-classical)'), 
        # ("gbm",'gbm(semi-classical-retraining)'),
        ("classical", "tick(ex)"),
        ("classical", "quote(ex)"),
        ("classical", "lr(ex)"),
        ("classical", "emo(ex)"),
        ("classical", "clnv(ex)"),
        (
            "classical",
            "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)",
        ),
    ]
]


In [6]:
results_data


gbm                            \
         gbm(classical) gbm(classical-retraining)   
index                                               
39342191             -1                        -1   
39342190             -1                        -1   
39342189              1                         1   
39342188             -1                        -1   
39342187             -1                        -1   
...                 ...                       ...   
49203742             -1                        -1   
49203743              1                         1   
49203744              1                         1   
49203745              1                         1   
49203746              1                         1   

                                                                     \
         gbm(classical-size-retraining) gbm(classical-size) gbm(ml)   
index                                                                 
39342191                             -1                  -1      -1   
39342190                             -1                  -1      -1   
39342189                              1                   1       1   
39342188                             -1                  -1      -1   
39342187                             -1                  -1      -1   
...                                 ...                 ...     ...   
49203742                              1                   1      -1   
49203743                              1                   1       1   
49203744                              1                   1       1   
49203745                              1                   1       1   
49203746                              1                   1       1   

                                                 \
         gbm(ml-retraining) gbm(semi-classical)   
index                                             
39342191                 -1                  -1   
39342190                 -1                  -1   
39342189                  1                   1   
39342188                 -1                  -1   
39342187                 -1                  -1   
...                     ...                 ...   
49203742                 -1                  -1   
49203743                  1                   1   
49203744                  1                   1   
49203745                  1                   1   
49203746                  1                   1   

                                        classical                           \
         gbm(semi-classical-retraining)  tick(ex) quote(ex) lr(ex) emo(ex)   
index                                                                        
39342191                             -1       1.0      -1.0   -1.0    -1.0   
39342190                             -1      -1.0      -1.0   -1.0    -1.0   
39342189                              1       1.0       1.0    1.0     1.0   
39342188                             -1      -1.0      -1.0   -1.0    -1.0   
39342187                             -1       1.0      -1.0   -1.0    -1.0   
...                                 ...       ...       ...    ...     ...   
49203742                             -1      -1.0       1.0    1.0     1.0   
49203743                              1      -1.0       1.0    1.0    -1.0   
49203744                              1      -1.0      -1.0   -1.0    -1.0   
49203745                              1      -1.0       1.0    1.0     1.0   
49203746                              1       1.0       1.0    1.0     1.0   

                   \
         clnv(ex)   
index               
39342191     -1.0   
39342190     -1.0   
39342189      1.0   
39342188     -1.0   
39342187     -1.0   
...           ...   
49203742      1.0   
49203743      1.0   
49203744     -1.0   
49203745      1.0   
49203746      1.0   

                                                                                        
         trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)  
i

### Robustness Checks

In [7]:
# prepare columns for printing
X_print["ttm"] = (
    X_print["EXPIRATION"].dt.to_period("M")
    - X_print["QUOTE_DATETIME"].dt.to_period("M")
).apply(lambda x: x.n)

X_print["year"] = X_print["QUOTE_DATETIME"].dt.year

bins_tradesize = [-1, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
X_print["TRADE_SIZE_binned"] = pd.cut(
    X_print["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

# p. 38
bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
X_print["year_binned"] = pd.cut(X_print["year"], bins_years, labels=year_labels)

# p. 37
bins_ttm = [-1, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "<= 1",
    "(1-2]",
    "(2-3]",
    "(3-6]",
    "(6-12]",
    "> 12",
]
X_print["ttm_binned"] = pd.cut(X_print["ttm"], bins_ttm, labels=ttm_labels)

# Security type
# see 3.0a-mb-explanatory-data-analysis.ipynb
X_print["issue_type"] = X_print["issue_type"].map(
    {
        "0": "Stock option",
        "A": "Index option",
        "7": "Others",
        "F": "Others",
        "%": "Others",
        " ": "Others",
    }
)

# Moneyness p. 38
bins_myn = [-1, 0.7, 0.9, 1.1, 1.3, np.inf]
myn_labels = [
    "<= 0.7 c",
    "(0.7-0.9]",
    "(0.9-1.1]",
    "(1.1-1.3]",
    "> 1.3",
]
X_print["myn_binned"] = pd.cut(X_print["myn"], bins_myn, labels=myn_labels)

# mid p. 31 + extra category for unknowns
ask = X_print["ask_ex"]
bid = X_print["bid_ex"]
trade_price = X_print["TRADE_PRICE"]

mid = np.where(ask >= bid, (ask + bid) * 0.5, np.nan)
half_spread = np.where(ask >= bid, (ask - bid) * 0.5, np.nan)

prox_quotes = np.where(
    trade_price == mid,
    0,  # at mid
    np.where(
        ((mid - half_spread) < trade_price) & (trade_price < (mid + half_spread)),
        1,  # inside
        np.where(
            (trade_price == (mid - half_spread)) | ((mid + half_spread) == trade_price),
            2,  # at quotes
            np.where(
                (trade_price < (mid - half_spread))
                | ((mid + half_spread) > trade_price),
                3,
                4,
            ),
        ),
    ),
)  # outside + unclassifiable

bins_prox = [-np.inf, 0, 1, 2, 3, 4]
prox_labels = [
    "at mid",
    "inside",
    "at quotes",
    "outside",
    "unknown",
]

X_print["prox_q_binned"] = pd.cut(prox_quotes, bins_prox, labels=prox_labels)
X_print["mid"] = mid

# clean up empty buckets, as it causes empty grouping in result set generation
X_print["year_binned"] = X_print["year_binned"].cat.remove_unused_categories()
X_print["myn_binned"] = X_print["myn_binned"].cat.remove_unused_categories()
X_print["ttm_binned"] = X_print["ttm_binned"].cat.remove_unused_categories()
X_print["prox_q_binned"] = X_print["prox_q_binned"].cat.remove_unused_categories()

X_print["all"] = "all"

X_print.drop(
    columns=[
        "EXPIRATION",
        "QUOTE_DATETIME",
        "TRADE_SIZE",
        "ttm",
        "myn",
        "ask_ex",
        "bid_ex",
        "year",
    ],
    inplace=True,
)


In [8]:
X_print.head(20)


,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,prox_q_binned,mid,all
index,,,,,,,,,,,
39342191,-1,3.500000,P,Stock option,"(3,5]",2015,<= 1,(0.9-1.1],at quotes,3.675000,all
39342190,-1,6.380000,C,Stock option,"(0,1]",2015,<= 1,(0.9-1.1],inside,6.575000,all
39342189,1,0.130000,P,Others,"(0,1]",2015,<= 1,(0.7-0.9],at quotes,0.100000,all
39342188,-1,0.040000,C,Others,"(1,3]",2015,<= 1,(0.7-0.9],at quotes,0.070000,all
39342187,-1,0.400000,C,Others,"(3,5]",2015,<= 1,(0.9-1.1],inside,0.430000,all
39342186,1,1.650000,C,Others,"(3,5]",2015,(3-6],(0.9-1.1],inside,1.605000,all
39342185,1,0.350000,C,Stock option,"(0,1]",2015,<= 1,(0.7-0.9],unknown,NaN,all
39342184,1,10.340000,C,Stock option,>11,2015,<= 1,(1.1-1.3],inside,10.275000,all
39342183,-1,15.170000,C,Stock option,>11,2015,<= 1,(1.1-1.3],inside,15.275000,all


In [9]:
X_print = pd.concat([X_print, results_data], axis=1)


In [10]:
X_print.head()


,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,prox_q_binned,mid,...,"(gbm, gbm(ml))","(gbm, gbm(ml-retraining))","(gbm, gbm(semi-classical))","(gbm, gbm(semi-classical-retraining))","(classical, tick(ex))","(classical, quote(ex))","(classical, lr(ex))","(classical, emo(ex))","(classical, clnv(ex))","(classical, trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all))"
index,,,,,,,,,,,,,,,,,,,,,
39342191,-1,3.50,P,Stock option,"(3,5]",2015,<= 1,(0.9-1.1],at quotes,3.675,...,-1,-1,-1,-1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0
39342190,-1,6.38,C,Stock option,"(0,1]",2015,<= 1,(0.9-1.1],inside,6.575,...,-1,-1,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
39342189,1,0.13,P,Others,"(0,1]",2015,<= 1,(0.7-0.9],at quotes,0.100,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0
39342188,-1,0.04,C,Others,"(1,3]",2015,<= 1,(0.7-0.9],at quotes,0.070,...,-1,-1,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
39342187,-1,0.40,C,Others,"(3,5]",2015,<= 1,(0.9-1.1],inside,0.430,...,-1,-1,-1,-1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0


## Results Set Generation

In [11]:
LUT = {
    "Trade_Size(ex)->Quote(Best)->Depth(Best)->Quote(Ex)->Depth(Ex)->Rev_Tick(All)": "\gls{GBM}",
    "(Ex)": " (Ex)",
    "(Best)": " (Best)",
    "(Classical)": " (Classical)",
    "(Classical-Size)": " (Classical, Size)",
    "Rev_": "Rev. ",
    "Trade_Size": "Trade Size",
    "Depth": "Depth",
    "->": " $\\to$ ",
    "Lr": "\gls{LR}",
    "Emo": "\gls{EMO}",
    "Clnv": "\gls{CLNV}",
    "OPTION_TYPE": "Option Type",
    "_": "$\_",
    "Gbm": "\gls{GBM}",
}

LUT_INDEX = {
    "OPTION_TYPE": "Option Type",
    "issue_type": "Security Type",
    "TRADE_SIZE_binned": "Trade Size",
    "year_binned": "Year",
    "ttm_binned": "Time to Maturity",
    "myn_binned": "Moneyness",
    "prox_q_binned": "Location to Quote",
    "all":"All trades",
}


def cell_str(x):
    x = x.title()
    for orig, sub in LUT.items():
        x = x.replace(orig, sub)
    # title-case everything
    return x


def highlight_max(s, props=""):
    return np.where(s == np.nanmax(s.values), props, "")


In [12]:
def set_tex_style(styler, caption, label, bold_axis=1):
    res = styler.set_caption(caption)
        

    res = (res.apply(
            highlight_max, props="font-weight:bold;", axis=bold_axis
        )
        .format(precision=4, decimal=".", thousands=",", escape=False, hyperlinks=None)
        .format_index(cell_str, axis=0)
        .format_index(cell_str, axis=1)
        .to_latex(
            f"{label}.tex",
            siunitx=True,
            position_float="centering",
            hrules=True,
            clines="skip-last;data",
            label="tab:" + label,
            caption=caption,
            convert_css=True,
        )
    )
    return res


In [13]:
classifiers = results_data.columns.tolist()
criterions = list(LUT_INDEX)


In [14]:
results_data.columns.tolist()


[('gbm', 'gbm(classical)'),
 ('gbm', 'gbm(classical-retraining)'),
 ('gbm', 'gbm(classical-size-retraining)'),
 ('gbm', 'gbm(classical-size)'),
 ('gbm', 'gbm(ml)'),
 ('gbm', 'gbm(ml-retraining)'),
 ('gbm', 'gbm(semi-classical)'),
 ('gbm', 'gbm(semi-classical-retraining)'),
 ('classical', 'tick(ex)'),
 ('classical', 'quote(ex)'),
 ('classical', 'lr(ex)'),
 ('classical', 'emo(ex)'),
 ('classical', 'clnv(ex)'),
 ('classical',
  'trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)')]

In [15]:
X_print.head()

,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,prox_q_binned,mid,...,"(gbm, gbm(ml))","(gbm, gbm(ml-retraining))","(gbm, gbm(semi-classical))","(gbm, gbm(semi-classical-retraining))","(classical, tick(ex))","(classical, quote(ex))","(classical, lr(ex))","(classical, emo(ex))","(classical, clnv(ex))","(classical, trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all))"
index,,,,,,,,,,,,,,,,,,,,,
39342191,-1,3.50,P,Stock option,"(3,5]",2015,<= 1,(0.9-1.1],at quotes,3.675,...,-1,-1,-1,-1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0
39342190,-1,6.38,C,Stock option,"(0,1]",2015,<= 1,(0.9-1.1],inside,6.575,...,-1,-1,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
39342189,1,0.13,P,Others,"(0,1]",2015,<= 1,(0.7-0.9],at quotes,0.100,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0
39342188,-1,0.04,C,Others,"(1,3]",2015,<= 1,(0.7-0.9],at quotes,0.070,...,-1,-1,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
39342187,-1,0.40,C,Others,"(3,5]",2015,<= 1,(0.9-1.1],inside,0.430,...,-1,-1,-1,-1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [16]:
X_print[X_print["prox_q_binned"]=="unknown"]

,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,prox_q_binned,mid,...,"(gbm, gbm(ml))","(gbm, gbm(ml-retraining))","(gbm, gbm(semi-classical))","(gbm, gbm(semi-classical-retraining))","(classical, tick(ex))","(classical, quote(ex))","(classical, lr(ex))","(classical, emo(ex))","(classical, clnv(ex))","(classical, trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all))"
index,,,,,,,,,,,,,,,,,,,,,
39342185,1,0.35,C,Stock option,"(0,1]",2015,<= 1,(0.7-0.9],unknown,NaN,...,1,1,1,1,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
39342179,1,11.30,C,Stock option,"(5,11]",2015,> 12,> 1.3,unknown,NaN,...,1,1,1,1,1.0,-1.0,1.0,1.0,1.0,1.0
39342234,1,0.41,C,Others,"(5,11]",2015,<= 1,(0.9-1.1],unknown,NaN,...,1,1,1,1,1.0,-1.0,-1.0,-1.0,-1.0,1.0
39342231,-1,0.68,P,Stock option,"(3,5]",2015,<= 1,(0.7-0.9],unknown,NaN,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0
39342225,-1,0.03,C,Others,"(5,11]",2015,<= 1,(0.9-1.1],unknown,NaN,...,-1,-1,-1,-1,-1.0,-1.0,1.0,1.0,1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49202809,-1,1.35,C,Stock option,"(1,3]",2017,(3-6],(0.9-1.1],unknown,1.300,...,-1,-1,1,-1,-1.0,1.0,1.0,1.0,1.0,-1.0
49202810,-1,1.35,C,Stock option,"(0,1]",2017,(3-6],(0.9-1.1],unknown,1.300,...,-1,-1,1,-1,-1.0,1.0,1.0,1.0,1.0,-1.0
49203153,1,0.21,C,Others,"(1,3]",2017,(1-2],(0.9-1.1],unknown,0.200,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0


In [17]:
X_print.head().T

index,39342191,39342190,39342189,39342188,39342187
buy_sell,-1,-1,1,-1,-1
TRADE_PRICE,3.5,6.38,0.13,0.04,0.4
OPTION_TYPE,P,C,P,C,C
issue_type,Stock option,Stock option,Others,Others,Others
TRADE_SIZE_binned,"(3,5]","(0,1]","(0,1]","(1,3]","(3,5]"
year_binned,2015,2015,2015,2015,2015
ttm_binned,<= 1,<= 1,<= 1,<= 1,<= 1
myn_binned,(0.9-1.1],(0.9-1.1],(0.7-0.9],(0.7-0.9],(0.9-1.1]
prox_q_binned,at quotes,inside,at quotes,at quotes,inside
mid,3.675,6.575,0.1,0.07,0.43


## Accuracy Calculation

In [18]:
# FIXME: Find better approach
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

result_dfs = []

for criterion in tqdm(criterions):
    results = []
    for classifier in tqdm(classifiers):
        res = (
            X_print.groupby([criterion])[["buy_sell", classifier]]
            .apply(
                lambda x: accuracy_score(x["buy_sell"].astype("int8"), x[classifier])
            )
            .mul(100)
            .rename(classifier)
        )
#         acc_tot = accuracy_score(
#             X_print["buy_sell"].astype("int8"), X_print[classifier]
#         )

#         res.loc["all"] = acc_tot * 100

        res.index.name = LUT_INDEX.get(criterion)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1).T
    result_df.style.pipe(
        set_tex_style,
        caption=(f"long-tbd", "short-tbd"),
        label=f"{key.lower()}-{criterion.lower()}",
    )

    # store all result sets for later use
    result_dfs.append(result_df)


100%|██████████| 8/8 [02:38<00:00, 19.82s/it]


In [19]:
master = pd.concat(result_dfs, axis=1, keys=list(LUT_INDEX.values())).T


In [20]:
master


gbm                            \
                               gbm(classical) gbm(classical-retraining)   
Option Type       C                 62.889592                 65.781794   
                  P                 64.544991                 67.112672   
Security Type     Index option      56.308184                 56.614402   
                  Others            68.405530                 70.200752   
                  Stock option      61.866828                 65.027478   
Trade Size        (0,1]             61.886324                 64.923233   
                  (1,3]             62.455088                 65.420139   
                  (3,5]             62.683294                 65.649639   
                  (5,11]            65.941513                 68.272577   
                  >11               67.305363                 69.386769   
Year              2015              60.396806                 63.511360   
                  2016              63.737864                 66.222497   
                  2017              64.587031                 67.796378   
Time to Maturity  <= 1              64.456091                 66.953853   
                  (1-2]             64.596545                 67.148388   
                  (2-3]             62.979868                 65.494804   
                  (3-6]             61.803170                 65.104390   
                  (6-12]            61.849926                 65.082867   
                  > 12              55.148148                 60.880427   
Moneyness         <= 0.7 c          65.551797                 68.889840   
                  (0.7-0.9]         67.636359                 70.102678   
                  (0.9-1.1]         64.063302                 66.473410   
                  (1.1-1.3]         54.203956                 58.776086   
                  > 1.3             52.572955                 58.768938   
Location to Quote at mid            62.620494                 63.537872   
                  inside            62.674689                 64.943835   
                  at quotes         67.707589                 73.574666   
                  outside           68.640271                 72.558984   
                  unknown           71.086696                 76.151229   
All trades        all               63.662370                 66.403078   

                                                               \
                               gbm(classical-size-retraining)   
Option Type       C                                 73.577088   
                  P                                 74.375973   
Security Type     Index option                      58.684208   
                  Others                            77.549609   
                  Stock option                      72.731024   
Trade Size        (0,1]                             73.782177   
                  (1,3]                             74.408553   
                  (3,5]                             74.374525   
                  (5,11]                            73.690410   
                  >11                               73.596321   
Year              2015                              71.335358   
                  2016                              74.119968   
                  2017                              74.436152   
Time to Maturity  <= 1                              74.381920   
                  (1-2]                             74.301318   
                  (2-3]                             73.117570   
                  (3-6]                             72.901755   
                  (6-12]                            72.439952   
                  > 12                              71.093367   
Moneyness         <= 0.7 c                          74.141656   
                  (0.7-0.9]                         75.791937   
                  (0.9-1.1]                         74.309486   
                  (1.1-1.3]                         69.216210   
                

In [21]:
master.style.pipe(
    set_tex_style, caption=("master-long", "master-short"), label=f"{key}-master", bold_axis=0
)


## Effective Spread 🚧

In [22]:
classifiers


[('gbm', 'gbm(classical)'),
 ('gbm', 'gbm(classical-retraining)'),
 ('gbm', 'gbm(classical-size-retraining)'),
 ('gbm', 'gbm(classical-size)'),
 ('gbm', 'gbm(ml)'),
 ('gbm', 'gbm(ml-retraining)'),
 ('gbm', 'gbm(semi-classical)'),
 ('gbm', 'gbm(semi-classical-retraining)'),
 ('classical', 'tick(ex)'),
 ('classical', 'quote(ex)'),
 ('classical', 'lr(ex)'),
 ('classical', 'emo(ex)'),
 ('classical', 'clnv(ex)'),
 ('classical',
  'trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)')]

In [23]:
eff_dfs = []


def stats(x, classifier):

    nom = effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="nominal")
    rel = (
        effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="relative")
        * 100
    )

    # eff_spread_pred = effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="none")
    # eff_spread_true = effective_spread(x["buy_sell"], x["TRADE_PRICE"], x["mid"], mode="none")
    # wilcoxon_res  = wilcoxon(eff_spread_pred, eff_spread_true, nan_policy="omit", zero_method="zsplit")

    return pd.Series(
        {
            "nominal": nom,
            "rel": rel,
            # 'statistic':wilcoxon_res.statistic,
            # 'pvalue':wilcoxon_res.pvalue
        }
    )


for criterion in tqdm(criterions):
    results = []

    for classifier in tqdm(classifiers):
        res = X_print.groupby([criterion])[
            ["TRADE_PRICE", "mid", classifier, "buy_sell"]
        ].apply(stats, classifier)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1, keys=classifiers).T
    result_df.style.pipe(
        set_tex_style,
        caption=(f"long-tbd", "short-tbd"),
        label=f"{key.lower()}-{criterion.lower()}-eff-spread",
    )

    # store all result sets for later use
    eff_dfs.append(result_df)


100%|██████████| 8/8 [01:30<00:00, 11.33s/it]


In [24]:
master_eff_spread = pd.concat(eff_dfs, axis=1, keys=LUT_INDEX.values()).T


In [25]:
master_eff_spread


gbm                 \
                               gbm(classical)                  
                                      nominal            rel   
Option Type       C                  0.079020   8.840429e+00   
                  P                  0.068563  9.776834e+302   
Security Type     Index option       0.242608   2.844856e+00   
                  Others             0.049453   1.265646e+01   
                  Stock option       0.081498  3.234859e+302   
Trade Size        (0,1]              0.087384   7.953118e+00   
                  (1,3]              0.077321   8.345212e+00   
                  (3,5]              0.071600   8.613399e+00   
                  (5,11]             0.062259   1.136177e+01   
                  >11                0.057662   1.096564e+01   
Year              2015               0.076555   7.951793e+00   
                  2016               0.073366   9.320020e+00   
                  2017               0.075116   9.290428e+00   
Time to Maturity  <= 1               0.061956   1.140240e+01   
                  (1-2]              0.071410   5.580789e+00   
                  (2-3]              0.080289   4.635582e+00   
                  (3-6]              0.098481   4.483614e+00   
                  (6-12]             0.108710   3.883437e+00   
                  > 12               0.179696   3.627786e+00   
Moneyness         <= 0.7 c           0.050747   1.852819e+01   
                  (0.7-0.9]          0.051802   1.711482e+01   
                  (0.9-1.1]          0.069510   7.695573e+00   
                  (1.1-1.3]          0.136256   1.863887e+00   
                  > 1.3              0.179638   1.804424e+00   
Location to Quote at mid             0.000000   0.000000e+00   
                  inside             0.089070   6.004961e+00   
                  at quotes          0.059433   3.015861e+01   
                  outside            0.045646   1.331227e+01   
                  unknown            0.038264            inf   
All trades        all                0.074138   9.185910e+00   

                                                                         \
                               gbm(classical-retraining)                  
                                                 nominal            rel   
Option Type       C                             0.037355   6.206095e+00   
                  P                             0.031439  9.776834e+302   
Security Type     Index option                  0.188044   1.865985e+00   
                  Others                        0.022307   1.077755e+01   
                  Stock option                  0.037189  3.234859e+302   
Trade Size        (0,1]                         0.041896   5.401829e+00   
                  (1,3]                         0.035934   5.696724e+00   
                  (3,5]                         0.033431   5.913642e+00   
                  (5,11]                        0.027265   8.636587e+00   
                  >11                           0.026759   8.445091e+00   
Year              2015                          0.035366   5.340948e+00   
                  2016                          0.035568   6.793718e+00   
                  2017                          0.032071   6.463349e+00   
Time to Maturity  <= 1                          0.028562   8.464189e+00   
                  (1-2]                         0.036892   3.736659e+00   
                  (2-3]                         0.039919   2.776066e+00   
                  (3-6]                         0.045905   2.267324e+00   
                  (6-12]                        0.046535   1.892412e+00   
                  > 12                          0.083614   1.813531e+00   
Moneyness         <= 0.7 c                      0.011315   1.257894e+01   
                  (0.7-0.9]                     0.021853   1.283074e+01   
                  (0.9-1.1]                     0.036305   5.494555e+00   
                  (1.1-1.3]                     0.0495

In [26]:
master_eff_spread.style.pipe(
    set_tex_style,
    caption=("master-short", "master-long"),
    label=f"{key}-master-eff-spread",
    bold_axis=0,
)


## Change in Parenthesis 🅾️

In [27]:
master

gbm                            \
                               gbm(classical) gbm(classical-retraining)   
Option Type       C                 62.889592                 65.781794   
                  P                 64.544991                 67.112672   
Security Type     Index option      56.308184                 56.614402   
                  Others            68.405530                 70.200752   
                  Stock option      61.866828                 65.027478   
Trade Size        (0,1]             61.886324                 64.923233   
                  (1,3]             62.455088                 65.420139   
                  (3,5]             62.683294                 65.649639   
                  (5,11]            65.941513                 68.272577   
                  >11               67.305363                 69.386769   
Year              2015              60.396806                 63.511360   
                  2016              63.737864                 66.222497   
                  2017              64.587031                 67.796378   
Time to Maturity  <= 1              64.456091                 66.953853   
                  (1-2]             64.596545                 67.148388   
                  (2-3]             62.979868                 65.494804   
                  (3-6]             61.803170                 65.104390   
                  (6-12]            61.849926                 65.082867   
                  > 12              55.148148                 60.880427   
Moneyness         <= 0.7 c          65.551797                 68.889840   
                  (0.7-0.9]         67.636359                 70.102678   
                  (0.9-1.1]         64.063302                 66.473410   
                  (1.1-1.3]         54.203956                 58.776086   
                  > 1.3             52.572955                 58.768938   
Location to Quote at mid            62.620494                 63.537872   
                  inside            62.674689                 64.943835   
                  at quotes         67.707589                 73.574666   
                  outside           68.640271                 72.558984   
                  unknown           71.086696                 76.151229   
All trades        all               63.662370                 66.403078   

                                                               \
                               gbm(classical-size-retraining)   
Option Type       C                                 73.577088   
                  P                                 74.375973   
Security Type     Index option                      58.684208   
                  Others                            77.549609   
                  Stock option                      72.731024   
Trade Size        (0,1]                             73.782177   
                  (1,3]                             74.408553   
                  (3,5]                             74.374525   
                  (5,11]                            73.690410   
                  >11                               73.596321   
Year              2015                              71.335358   
                  2016                              74.119968   
                  2017                              74.436152   
Time to Maturity  <= 1                              74.381920   
                  (1-2]                             74.301318   
                  (2-3]                             73.117570   
                  (3-6]                             72.901755   
                  (6-12]                            72.439952   
                  > 12                              71.093367   
Moneyness         <= 0.7 c                          74.141656   
                  (0.7-0.9]                         75.791937   
                  (0.9-1.1]                         74.309486   
                  (1.1-1.3]                         69.216210   
                

In [28]:
base = master[[("classical","quote(ex)"),("classical", "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)"), ("classical", "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)")]]
revised = master[[("gbm","gbm(classical)"),("gbm","gbm(classical-size)"), ("gbm", "gbm(ml)")]]

In [34]:

def combine_results(revised: pd.DataFrame, base: pd.DataFrame) -> pd.DataFrame:
    """
    Generate print layout like in Grauer et al.

    https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290

    # see p. https://texdoc.org/serve/siunitx/0
    """
    # first, second layer of colum index
    c_1 = revised.columns.get_level_values(1)
    c_2 = ["nom"]
    midx = pd.MultiIndex.from_product([c_1, c_2])

    # copy data from revised add as (column, "nom")
    combo = pd.DataFrame(revised.values, index=revised.index, columns=midx)

    
    
    for i, mul_col in enumerate(combo.columns):

        combo[(mul_col[0], "pm")] = (combo[mul_col] - base.iloc[:,i]).round(2)
        combo.sort_index(axis=1, inplace=True)
        
    return combo


In [35]:
diff = combine_results(revised, base)

diff.style.to_latex(
            f"diff-{key}.tex",
            siunitx=True,
            position_float="centering",
            hrules=True,
            clines="skip-last;data",
            label=f"tab:diff-{key}",
            caption=(f"long-diff-{key}", f"short-diff-{key}"),
            convert_css=True,
        )

In [36]:
diff


gbm(classical)        gbm(classical-size)  \
                                          nom     pm                 nom   
Option Type       C                 62.889592   6.55           71.895259   
                  P                 64.544991   6.81           72.892594   
Security Type     Index option      56.308184   2.58           57.640799   
                  Others            68.405530   6.01           76.385942   
                  Stock option      61.866828   7.00           70.962294   
Trade Size        (0,1]             61.886324   6.60           72.589837   
                  (1,3]             62.455088   7.15           73.006333   
                  (3,5]             62.683294   6.95           72.766890   
                  (5,11]            65.941513   6.07           71.771961   
                  >11               67.305363   6.59           71.307368   
Year              2015              60.396806   5.52           69.028053   
                  2016              63.737864   6.17           72.560975   
                  2017              64.587031   8.23           73.018742   
Time to Maturity  <= 1              64.456091   7.15           72.770627   
                  (1-2]             64.596545   6.88           72.885430   
                  (2-3]             62.979868   5.88           71.797757   
                  (3-6]             61.803170   5.82           71.302394   
                  (6-12]            61.849926   5.44           71.222635   
                  > 12              55.148148   2.48           68.602077   
Moneyness         <= 0.7 c          65.551797   5.21           71.843131   
                  (0.7-0.9]         67.636359   7.27           74.290189   
                  (0.9-1.1]         64.063302   6.98           72.990099   
                  (1.1-1.3]         54.203956   4.27           66.292506   
                  > 1.3             52.572955   3.84           63.007743   
Location to Quote at mid            62.620494  12.66           72.155374   
                  inside            62.674689   5.60           68.799095   
                  at quotes         67.707589   7.72           86.707454   
                  outside           68.640271   4.55           84.900871   
                  unknown           71.086696   9.94           85.707496   
All trades        all               63.662370   6.67           72.360838   

                                        gbm(ml)         
                                  pm        nom     pm  
Option Type       C             5.04  73.624730   6.77  
                  P             5.63  74.593715   7.33  
Security Type     Index option -0.88  58.100126  -0.42  
                  Others        6.51  77.551217   7.67  
                  Stock option  4.93  72.919301   6.89  
Trade Size        (0,1]         4.01  74.242395   5.66  
                  (1,3]         4.35  74.813673   6.16  
                  (3,5]         3.94  74.557255   5.73  
                  (5,11]        8.57  73.383853  10.18  
                  >11           6.88  73.084679   8.66  
Year              2015          5.73  71.423694   8.13  
                  2016          5.59  74.235176   7.27  
                  2017          4.53  74.603733   6.11  
Time to Maturity  <= 1          5.80  74.560803   7.59  
                  (1-2]         4.97  73.865438   5.95  
                  (2-3]         4.12  72.946987   5.26  
                  (3-6]         4.03  72.719581   5.45  
                  (6-12]        3.95  73.093080   5.82  
                  > 12          4.01  72.039005   7.44  
Moneyness         <= 0.7 c      7.85  72.438812   8.45  
                  (0.7-0.9]     6.65  75.357081   7.71  
                  (0.9-1.1]     5.00  74.414466   6.43  
                  (1.1-1.3]     4.66  70.268926   8.64  
                  > 1.3         3.05  70.550826  10.60  
Location to Quote at mid        6.24  74.659713   8.74  
                  inside        4.35  70.688189   6.24  
  